## Исследование данных о российском кинопрокате



**Цель исследования** — проверить гипотезы:
1. Выяснить за какой год было зафиксированно наибольшее число релизов.
2. Изучить влияет ли возрастное ограничение на рейтинг и кассовый сбор фильма, как и в какие года какие показатели. 
3. Какова окупаемость, рейтинги у фильмов, финансируемых государственным бюджетом.
**Ход исследования**
 1. Обзор данных.
 2. Предобработка данных.
 3. Расчет и визуализация необходимых данных.
 4. Проверка гипотез.

### Обзор данных

In [1]:
import pandas as pd

In [2]:
mkrf_movies = pd.read_csv(r"C:\Users\yana\Desktop\проекты\минкульт рф\mkrf_movies.csv")
mkrf_shows = pd.read_csv(r"C:\Users\yana\Desktop\проекты\минкульт рф\mkrf_shows.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\yana\\Desktop\\проекты\\минкульт рф\\mkrf_movies.csv'

In [ ]:
def prepare(df): #фунция предобработки данных
    display(df.head())
    display(df.info())
    display(df.isna().sum())
    display(df.duplicated().sum())

In [ ]:
prepare(mkrf_movies)

In [ ]:
prepare(mkrf_shows)

In [ ]:
mkrf_movies['puNumber'] = pd.to_numeric(mkrf_movies['puNumber'], errors='coerce')
mkrf_movies['puNumber'] = mkrf_movies['puNumber'].fillna(0)
mkrf_movies['puNumber'] = mkrf_movies['puNumber'].astype('int')

In [ ]:
movies = mkrf_movies.merge(mkrf_shows, on='puNumber', how='left')

### Предобработка данных

In [ ]:
movies['ratings'] = pd.to_numeric(movies['ratings'], errors='coerce')

In [ ]:
movies['age_restriction'].unique()
def short_age(age):
    if '18+' in age:
        return '18'
    elif '16+' in age:
        return '16'
    elif '12+' in age:
        return '12'
    elif '6+' in age:
        return '6'
    elif '0+' in age:
        return '0' 
movies['age_restriction'] = movies['age_restriction'].apply(short_age)
movies['age_restriction'] = movies['age_restriction'].astype('int')
movies['age_restriction'].unique()

**Тип данных** изменила тип данных изначально в столбце id фильма для дальнейшего объединения столбцов, перевела рейтинг в float, тк были выбивающиеся значения, ограничесния по возрасту видоизменила и перевела в целочисленные для дальнейшего удобства и снижения занимаемой памяти.

In [ ]:
movies = movies.dropna(subset=['director'])

**Пропуски** были заполнены частично на предыдущем этапе, чтобы изменить тип данных. Удалила строки с пропусками в графе режиссер, тк дальше нужно анализировать данные о режиссерах и их жанрах.

In [ ]:
movies['title'].duplicated().sum()
movies.groupby('title').agg(count=('title','count'))[lambda x : x['count'] > 1] 
#фнукция для подсчета дублированных названий фильмов в отдельный словарь.

In [ ]:
movies.groupby('title').agg(count=('title','count'))[lambda x : x['count'] > 1]

In [ ]:
data_copy = movies.copy()

In [ ]:
idx = data_copy[data_copy.duplicated(subset='title', keep=False) & (data_copy['box_office'].isna() == False)].index
data_copy.loc[idx]

In [ ]:
len(set(idx) - set(movies.index))

Дубли в названиях присутствуют в незначительном количесвте, но содержат в себе ту или иную важную информацию, поэтому удалять их не будем.

In [ ]:
movies['type'] = movies['type'].str.strip()
movies['type'].unique()

In [ ]:
movies['film_studio'].unique()

In [ ]:
movies['production_country'].unique()

In [ ]:
movies['director'].unique()

In [ ]:
movies['financing_source'].unique()

In [ ]:
movies['genres'].unique()

In [ ]:
movies['budget'].unique() 
movies['refundable_support'].unique()
movies['nonrefundable_support'].unique()

refundable_support и nonrefundable_support входят в budget. есть пропуски  в графах невозвращаемого объема(что может быть), есть пропуски в графе бюджет. Не знала заполнять или нет, провела проект до конца, поняла что с пропусками значеня получились более похожими на правду, тк если заполнять их 0 или -1, конечные данные по вычислениям искажаются.

In [ ]:
movies['start_year'] = pd.to_datetime(movies['show_start_date'], format='%Y-%m-%dT%H:%M:%S')
movies['start_year'] = movies['start_year'].dt.year #год рубликации

In [ ]:
def first_name(name): #функция вычленияет значения до запятой, то есть только первое имя или жанр.
    try:
        return name.partition(',')[0]
    except:
        return name
movies['first_director'] = movies['director'].apply(first_name)
movies['first_genre'] = movies['genres'].apply(first_name)   

### Расчет и визуализация необходимых данных.

In [ ]:
movies['part_of_budget'] = (movies['refundable_support'] + movies['nonrefundable_support']) / movies['budget'] 
#тк возвратная и невозвратная это переменные, означающие фин помощь от государства, 
#то их сумму делим на общий бюджет и получаем долю господдержки.
gos_movies = movies[movies['budget'] >= 0] #для проверки

In [ ]:
for year in range (2010, 2020):
    display((movies[movies['start_year'] == year]['box_office'].isna() == False).mean()) 
#высчитана доля заполненных фильмов по каждому году

In [ ]:
movies.pivot_table(index = {'start_year'}, values='box_office', aggfunc='sum').plot(kind='barh') 
#наглядный график
sbor = movies.pivot_table(index = {'start_year'}, values='box_office', aggfunc=['sum', 'mean', 'median']) 
#сводная таблица с суммой, медианой и средним арифметическим.
sbor.columns = ['summa','mean_summa', 'median_summa']
sbor_new = sbor.sort_values(by = 'summa') 
min_sbor = sbor_new.iloc[0] #год с минимальным сбором
max_sbor = sbor_new.iloc[-1] #год с максимальным сбором

In [ ]:
with pd.option_context('display.float_format','{:,.0f}'.format):
    display(sbor) #расчеты в предыдыущем пункте
    
type(sbor)

In [ ]:
age_ristrict = movies.query('start_year >=2015')
age_ristrict.pivot_table(index='start_year', columns='age_restriction', values='box_office', aggfunc={'box_office' :'sum'}).plot(kind='bar', figsize=(15,5))
age_ristrict.pivot_table(index={'start_year', 'age_restriction'}, values='box_office', aggfunc={'box_office' :'sum'})

### Проверка гипотез

In [ ]:
gos_movies = movies[movies['budget'] >= 0] #для проверки
gos_movies #ознакомилась с гос фильмами

In [ ]:
payback = gos_movies[gos_movies['box_office'] > gos_movies['budget']]
payback_fin = len(payback)/len(gos_movies)
payback_fin #окупаемость 25%

In [ ]:
gos_movies['ratings'].describe()

In [ ]:
gos_movies['first_genre'].value_counts()

In [ ]:
gos_movies['age_restriction'].value_counts()

In [ ]:
gos_movies.pivot_table(index='start_year', values='ratings', aggfunc='median').plot(); #зависимость года выпуска и рейтинга

In [ ]:
gos_movies.pivot_table(index='start_year', values='budget', aggfunc='median').plot(); #зависимость года выпуска и бюджета

In [ ]:
payback.pivot_table(index='start_year', values='ratings', aggfunc='median').plot(); #зависимость года выпуска и рейтинга

In [ ]:
payback.pivot_table(index='start_year', values='budget', aggfunc='median').plot(); #зависимость года выпуска и бюджета

In [ ]:
correlyatia = payback['box_office'].corr(movies['budget'])

In [ ]:
for year in range(2015, 2020):
    ganre_years = gos_movies[gos_movies['start_year'] == year] 
    b = ganre_years['first_genre'].value_counts().head(5)
    print(b)

In [ ]:
for year in range(2015, 2020):
    ganre_years_n = payback[payback['start_year'] == year] 
    c = ganre_years_n['first_genre'].value_counts().head(5)
    print(c)

**Общий вывод:**
- В ходе исследования было выполнено ознакомление с данными, их предобработка, сцелью улучшить их качество и сделать более точные выводы. Далее производились необходимые расчеты. Данные представлены в виде расчетов и визуальных структур с целью более комплексного и понятного восприятия.
- Из года в год доля фильмов с известным бюджетом стала больше: с 10% поднялась до 70% с 2015-20117 годах, и подупала до 50% в 2018-2019 гг.
- Сборы и возрастные ограничения: за 2015 год больше всего принесли прибыль  фильмы с возрастным ограниченем 12+, в следующем 2016 году кассовые сборы подтянулись и прибыльными также оказались помимо 12+, фильмы 6+ И 18+, за 2017 год более прибыльными стали только 6+ и 18+, за следующие 2 года(2018 и 2019 соответственно) киностудии подтянулись и в прибыльные опять вышли фильмы с рейтнгом 12+, помимо 6 и 18. Таким образом, наиболее окупаемые фильмы в средней возрастной категории.
- **Фильмов, спосируемых государством чуть больше 300 из представленных данных**, по ним мы можем сказать следующее:
- **Окупаемость фильмов, спонсированных государством, чуть больше 25%.** За окупаемость считается кассовые сборы, превысившие бюджет фильма.
- Фильмы с господдержкой имеют **чаще всего рейтинг чуть больше 6**, что является не самым лучшим показателем. Среднее арифметическое примерно такое же.
- Чаще всего финансируемыми государством снимали фильмы в **жанре драма, комедия, мультфильм, мелодрама и триллер.**
- Больше всего фильмов снято в **средней возрастной категории**.
- Чаще всего **рейтинг с каждым годом становится немного меньше.** В период, начиная с 2017, рейтинг вышел на плато. Хотя финансирование после таких фильмов только увеличивалось.
- Если рассматривать **окупившиеся фильмы, то можем сказать, что денег на них тратят гораздо больше, а жанры остаются все те же.**
- **Самые удачные фильмы были все равно из тройки лидеров: драма, комедия или мультфильм. при этом окупаются из них чаще всего комедии.**